In [1]:
#!/bin/python3

In [2]:
#Tokanisation is nothing but making it encripted and in this we will learn byte pair encoding insted of 
#conventional way of just giving out character encoding method 

In [3]:
# so how do the computer understand a character the answer is unicode 
#which has around 160000 character in itself including all the emojies and other languages 
#here is a representation
ord("4")# it only takes in one character insted of a whole string so when ur trying to use string use a for loop like this 
[ord(x) for x in "something"]

[115, 111, 109, 101, 116, 104, 105, 110, 103]

In [4]:
#but in later future get to know that we can directly use the encoding done in the bytes like utf-8 or utf-16 
#but here is the famous utf-8 enclding schema
list("something".encode("utf-8"))
# see its the same as above 

[115, 111, 109, 101, 116, 104, 105, 110, 103]

In [5]:
#But we can't just feed this into the attention trasformer casue the seq-len will be very long for short or some long data 
#and will not be as effective and the network wil become long for just small sets of data and it will not learn as 
# it will learn from a set of words insted of alphabets encoding 
# and thats where byte-pair encoding comes at 
# eg we have a string "aabbccbbcccaa" its vocab size is 3 cause there are only a,b,c and seq_len/context-len is 13 
# we can make this thing/seq-len shorter by making the vocab size bigger here is an example
# Z= aa,X=bb,Y=cc
#now the string is = "ZXYZYZ" but the vocab size is a,b,c,X,Y,Z
#we can make it even more shorter if we see some common pairs 
#O=YZ
# "ZXOO" see the seq_len/context-len became shorter from 13 to 4 but the vocab size is a,b,c,X,Y,Z,O but this traid off is ok 


In [127]:
#here's how to do it 
# first we take data from somewhere 
with open("input.txt","r",encoding="utf-8")  as f:
    texts = f.read()
data = texts[:1000]
print(f"vocab of data : {len(set(data))}")

vocab of data : 46


In [128]:
tokens= data.encode("utf-8")
tokens = list(map(int,tokens))
tokens[:5]

[70, 105, 114, 115, 116]

In [129]:
#now we'll have to make pair encoding of these binary utf-8 representation
def get_stat(data):
    output = {}
    for pair in zip(data,data[1:]):
        output[pair] = output.get(pair,0)+1
    return output
stats =get_stat(tokens)
# sorted(((v,k) for k,v in stats.items()),reverse=True)

In [130]:
top_pair = max(stats,key=stats.get)
(bytes(top_pair)).decode("utf-8"),top_pair

('e ', (101, 32))

In [131]:
def merge(data,pair,idx):
    new_data = []
    i = 0
    while i<len(data):
        if i< len(data)-1 and pair[0]==data[i] and pair[1]==data[i+1]:
            new_data.append(idx)
            i+=2
        else:
            new_data.append(data[i])
            i+=1
    return new_data


In [135]:
max(tokens)

122

In [142]:
vocab_size =  200
num_merge = vocab_size-max(tokens)
ids = list(tokens)
merges = {}
for i in range(num_merge):
    stats = get_stat(ids)
    pair = max(stats,key=stats.get)
    idx = 123+i
    ids = merge(ids,pair,idx)
    merges[pair]=idx


In [143]:
# decoding it 
vocab= {idx: bytes([idx]) for idx in range(200)}
for (p0,p1),idx in merges.items():
    vocab[idx] =vocab[p0]+vocab[p1]
def decode(ids):
    tokens = b"".join(vocab[ix] for ix in ids)
    text = tokens.decode("utf-8")
    return text
decode([119])

'w'

In [144]:
#Encoding
def encoding(ids):
    tokens = list(ids.encode("utf-8"))
    while len(tokens) >=2:
        stats = get_stat(tokens)
        pair = min(stats,key=lambda p: merges.get(p,float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens,pair,idx)
    return tokens
encoding("hemlo")

[104, 101, 109, 108, 111]

In [145]:
[decode([p0,p1]) for (p0,p1),v in merges.items()]

['e ',
 'th',
 't ',
 's ',
 ', ',
 'en',
 'it',
 'r ',
 ':\n',
 'd ',
 '\n\n',
 'an',
 'ou',
 'iti',
 'itiz',
 'itizen',
 're',
 'ir',
 'irs',
 'the',
 '.\n\n',
 'll',
 'ar',
 'to',
 'Firs',
 'Citizen',
 'Citizen:\n',
 'y ',
 'pe',
 'es',
 'us ',
 'the ',
 'n ',
 'First ',
 'First Citizen:\n',
 'or',
 'we ',
 'ak',
 've',
 'to ',
 'no',
 'is ',
 'ic',
 'peak',
 'All',
 'All:\n',
 '.\n\nFirst Citizen:\n',
 'ol',
 'kno',
 'know',
 'ie',
 ': ',
 'or ',
 'speak',
 'are ',
 'll ',
 'olve',
 'mi',
 'f ',
 'le',
 "'t",
 'our ',
 'in',
 'on',
 'y, ',
 'ec',
 'go',
 'su',
 'ld ',
 'bu',
 'er',
 'ven',
 'in ',
 'for ',
 'ore ',
 'pr',
 'ce',
 'ed ']